In [14]:
import re
import os 
import sys 
import scipy 
import pickle 
import numpy as np
from tqdm import tqdm 
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split 
sys.path.append('/Users/lei/home/studyhall/smart-earth-sensing/office-demo-project/lib') 

import warnings 
warnings.filterwarnings('ignore')
 
import torch 
from torch.nn.functional import one_hot 
 
from spec import * 
from preprocessing import *
from h5reader import * 
from audio_augmentation import * 
from dataset import Audio_Dataset 
from visualize import * 

In [2]:
# with open('/Users/lei/home/studyhall/smart-earth-sensing/office-demo-project/data/walk_run_spec.txt', 'r') as f:
#     specs = [Spec(re.sub("\n| ", "", line).split(',')) for line in f.readlines()]

In [3]:
# dir_path = '/Users/lei/home/studyhall/smart-earth-sensing/office-demo-project/data/corpus'
# feature_set = []
# label_set = []
# for spec in specs:
#     print(spec) 
#     for time in spec.duration: 
#         for path in os.listdir(dir_path):
#             if path[42:46] == time:
#                 raw, _, _ = read_data(dir_path + '/' + path, [20], [21]) 
#                 samples4halfsec = list(divide_chunks(raw, 500))
#                 feature_set += samples4halfsec
#                 for i in range(0, np.shape(samples4halfsec)[0]):
#                     label_set.append(spec.label)

In [4]:
# sr = 1000 
# augmented_featrue_set = []
# augmented_label_set = [] 
# for i in tqdm(range(0, len(feature_set))):
#     audio = feature_set[i] 
#     augmented_featrue_set += [audio, 
#                             shift_pitch(audio=audio, sr=sr, pitch_factor=1.5), 
#                             shift_time(audio=audio, shift_max=50, shift_direction='rand'), 
#                             noise_injection(audio=audio, noise_factor=0.2)] 
#     augmented_label_set += [label_set[i] for j in range(0, 4)] 

In [5]:
augmented_feature_set = []
augmented_label_set = [] 
with open('../pk_files/augmented_corpus.pkl', 'rb') as f: 
    (augmented_feature_set, augmented_label_set) = pickle.load(f)
     
print("augmented featrure shape: {}".format(np.shape(augmented_feature_set)))
print("respective label shape: {}".format(np.shape(augmented_label_set))) 

augmented featrure shape: (31200, 500, 1)
respective label shape: (31200,)


In [9]:
features = np.reshape(np.array([raw2mfcc(raw=raw) for raw in tqdm(augmented_feature_set)]), (31200, 1, 26, 64))
labels = np.array(augmented_label_set)

100%|██████████| 31200/31200 [02:27<00:00, 211.14it/s]


In [15]:
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3)
X_train = [torch.tensor(sample, dtype=torch.float32) for sample in X_train]
X_val = [torch.tensor(sample, dtype=torch.float32) for sample in X_val]
y_train = [one_hot(torch.tensor(sample, dtype=torch.long), num_classes=3).to(torch.float32) for sample in y_train]
y_val = [one_hot(torch.tensor(sample, dtype=torch.long), num_classes=3).to(torch.float32) for sample in y_val]

In [16]:
data_train = Audio_Dataset(X_train, y_train, len(y_train))
data_val = Audio_Dataset(X_val, y_val, len(y_val)) 

In [17]:
with open('../data/train_val_dataset.pkl', 'wb') as f:
    pickle.dump((data_train, data_val), f)